In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import *
import math
from scipy import stats

In [3]:
# Importing data from csv files

rail_dist = pd.read_csv('Rail_dist_in_mm.csv')
trig_angle = pd.read_csv('angle_trig_inmeter.csv')
vink_angle = pd.read_csv('Angle_gen.csv')
string_laser = pd.read_csv('L_string_laser.csv') 
string_tape = pd.read_csv('L_string_tape.csv')
dia = pd.read_csv('ball_radius.csv')

rail_dist = rail_dist.to_numpy()
trig_angle = trig_angle.to_numpy()
vink_angle = vink_angle.to_numpy()
string_laser = string_laser.to_numpy()
string_tape = string_tape.to_numpy()
dia = dia.to_numpy()

In [4]:
def error_probagation(f, PAR,SIGMA):
    "GENERAL ERROR PROBAGATION"
    led = 0
    for i in range(len(PAR)):
        led += ((f.diff(PAR[i]) * SIGMA[i])**2)
    return sqrt(led)

In [5]:
# Rail width

mean_rail = np.mean(rail_dist)
error_rail = np.std(rail_dist)/np.sqrt(len(rail_dist))

print('Rail width = ', mean_rail, 'mm' , '±', round(error_rail, 2), 'mm')

Rail width =  10.05 mm ± 0.02 mm


In [6]:
# Trigonometric angle
from sympy.stats import Arcsin, density


hyp_mean = np.mean(trig_angle[:, 0])
kat_mean = np.mean(trig_angle[:, 1])
hyp_error = np.std(trig_angle[:, 0])/np.sqrt(len(trig_angle[:, 0]))
kat_error = np.std(trig_angle[:, 1])/np.sqrt(len(trig_angle[:, 1]))
angle_t = np.arcsin(kat_mean/hyp_mean)

# Define symbolic variables
a, b, da, db = symbols('a, b, sigma_a, sigma_b')

# Create the expression arcsin(a/b)
theta = asin(a/b)

par = [a, b]
sig = [da, db]

sig_theta = error_probagation(theta, par, sig)
ftheta = lambdify((a, b), theta)
fsig_theta = lambdify((a, b, da, db), sig_theta)

print("Angle theta:", ftheta(kat_mean, hyp_mean)*(180/np.pi))
print("Error on angle theta:", fsig_theta(kat_mean, hyp_mean, kat_error, hyp_error)*(180/np.pi))

Angle theta: 14.912937331336302
Error on angle theta: 0.2653079841934492


In [7]:
# Vinkelmåler angle

vink_norm = (vink_angle[:, :2])
vink_rev = (vink_angle[:, 3:5])

mean_norm = np.mean(vink_norm, axis=1)[:, np.newaxis]
mean_rev = np.mean(vink_rev, axis=1)[:, np.newaxis]

error_norm = np.std(mean_norm)/np.sqrt(len(mean_norm)) 
error_rev = np.std(mean_rev)/np.sqrt(len(mean_rev))

MEAN_norm = np.mean(mean_norm)  
MEAN_rev = np.mean(mean_rev) 

array_MEAN = np.hstack((MEAN_norm, MEAN_rev))


# Putting the two angle measurements together
MEAN_ang = (MEAN_norm/error_norm**2 + MEAN_rev/error_rev**2)/(1/error_norm**2 + 1/error_rev**2)
error_ang = np.sqrt(1/(1/error_norm**2 + 1/error_rev**2))
chi2_ang = np.sum((array_MEAN - MEAN_ang)**2/error_ang**2)
N_freedom = len(array_MEAN) - 1
chi2_prob = 1 - stats.chi2.cdf(chi2_ang, N_freedom) 


print('Angle = ', MEAN_ang, '±', round(error_ang, 2))
print('Chi2 = ', chi2_ang)
print('P-value = ', chi2_prob)

Angle =  13.711190053285971 ± 0.07
Chi2 =  0.17753863152382643
P-value =  0.6734970906060582


In [8]:
# String length

mid_lod = string_tape[:, -1]/2
length_tape_in = string_tape[:, 0]
length_tape_fin = string_tape[:, 1]


m_lti = np.mean(length_tape_in)/100   # initial length of string tape
m_ltf = np.mean(length_tape_fin)/100  # final length of string tape
er_lti = np.std(length_tape_in)/np.sqrt(len(length_tape_in))/100 
er_ltf = np.std(length_tape_fin)/np.sqrt(len(length_tape_fin))/100


length_with_lod = length_tape_in + mid_lod
mean_length = np.mean(length_with_lod)/100
error_length = np.std(length_with_lod)/np.sqrt(len(length_with_lod))/100


print("Mean length of string initial:", m_lti, "m")
print("Mean length of string final:", m_ltf, "m")
print("Error on length of string initial:", er_lti, "m")
print("Error on length of string final:", er_ltf, "m")
print("\n")

print("Mean length of string with lod:", mean_length, "m")
print("Error on length of string with lod:", error_length, "m \n")

weighted_mean_tape = (m_lti/er_lti**2 + m_ltf/er_ltf**2) / (1/er_lti**2 + 1/er_ltf**2)
weighted_error_tape = np.sqrt(1 / (1/er_lti**2 + 1/er_ltf**2))

chi2_length_tape = (m_lti - weighted_mean_tape)**2/er_lti**2 + (m_ltf - weighted_mean_tape)**2/er_ltf**2
N_freedom = 2 - 1
chi2_prob_tape = 1 - stats.chi2.cdf(chi2_length_tape, N_freedom)
print("Weighted mean = ", weighted_mean_tape, "+-", weighted_error_tape)
print('Chi2 = ', chi2_length_tape)
print('P-value = ', chi2_prob_tape)

Mean length of string initial: 1.827475 m
Mean length of string final: 1.824375 m
Error on length of string initial: 0.002557922741210145 m
Error on length of string final: 0.0009416574483324621 m


Mean length of string with lod: 1.8420250000000002 m
Error on length of string with lod: 0.002557739480478834 m 

Weighted mean =  1.8247449789695056 +- 0.000883680141306715
Chi2 =  1.2934595162985287
P-value =  0.2554113834001378


In [9]:
# String length with laser

length_laser_in = string_laser[:, 0]
length_laser_fin = string_laser[:, 1]

# Mean and error on length of string initial with laser (Without lod)
m_lli = np.mean(length_laser_in)
e_lli = np.std(length_laser_in)/np.sqrt(len(length_laser_in))
# Mean and error on length of string final with laser (Without lod)
m_llf = np.mean(length_laser_fin)
e_llf = np.std(length_laser_fin)/np.sqrt(len(length_laser_fin))

# Mean and error on length of string initial with laser (With lod)
length_with_lod_laser = length_laser_in + mid_lod/100
mean_length_laser = np.mean(length_with_lod_laser)
error_length_laser = np.std(length_with_lod_laser)/np.sqrt(len(length_with_lod_laser))

# Without lod
print("Mean length of string initial without lod:", m_lli, "m")
print("Error on length of string initial without lod:", e_lli, "m")
print("Mean length of string final without lod:", m_llf, "m")
print("Error on length of string final without lod:", e_llf, "m")
print("\n")

# With lod
print("Mean length of string initial with laser:", mean_length_laser, "m")
print("Error on length of string initial with laser:", round(error_length_laser, 4), "m \n")

weighted_mean_laser = (m_lli/e_lli**2 + m_llf/e_llf**2) / (1/e_lli**2 + 1/e_llf**2)
weighted_error_laser = np.sqrt(1 / (1/e_lli**2 + 1/e_llf**2))

chi2_length_laser = (m_lli - weighted_mean_laser)**2/e_lli**2 + (m_llf - weighted_mean_laser)**2/e_llf**2
N_freedom = 2 - 1
chi2_prob_laser = 1 - stats.chi2.cdf(chi2_length_laser, N_freedom)
print("Weighted mean = ", weighted_mean_laser, "+-", weighted_error_laser)
print('Chi2 = ', chi2_length_laser)
print('P-value = ', chi2_prob_laser)

Mean length of string initial without lod: 1.82425 m
Error on length of string initial without lod: 0.00021650635094608582 m
Mean length of string final without lod: 1.824 m
Error on length of string final without lod: 0.0011726039399558468 m


Mean length of string initial with laser: 1.8388 m
Error on length of string initial with laser: 0.0002 m 

Weighted mean =  1.8242417582417583 +- 0.0002129076568131373
Chi2 =  0.043956043955996324
P-value =  0.8339354140886406


In [10]:
length_array = np.hstack((mean_length_laser, mean_length))
error_length_array =  np.hstack((error_length_laser, error_length))

Weighted_mean = np.sum(length_array/error_length_array**2) / np.sum(1/error_length_array**2)
Weighted_error = np.sqrt(1 / np.sum(1/error_length_array**2))

chi2_length = (length_array[0] - Weighted_mean)**2/error_length_array[0]**2 + (length_array[1] - Weighted_mean)**2/error_length_array[1]**2
N_freedom = 2 - 1
chi2_prob = 1 - stats.chi2.cdf(chi2_length, N_freedom)
print("weighted mean = ", Weighted_mean, "+-", Weighted_error)
print('Chi2 = ', chi2_length)
print('P-value = ', chi2_prob)

weighted mean =  1.8388240061162078 +- 0.00022067462625767774
Chi2 =  1.5779816513763647
P-value =  0.20905166000727826


In [11]:
ball_dia = dia[:, 0]
ball_dia_mean = np.mean(ball_dia)
ball_dia_error = np.std(ball_dia)/np.sqrt(len(ball_dia))

print("Mean ball diameter:", ball_dia_mean, "mm")
print("Error on ball diameter:", round(ball_dia_error, 2), "mm")

Mean ball diameter: 14.875 mm
Error on ball diameter: 0.02 mm


In [12]:
# Calculating ∆theta and error on ∆theta

a_norm, a_rev = symbols('a_norm, a_rev')
theta = symbols('theta')
da_norm, da_rev = symbols('sigma_anorm, sigma_arev')
dtheta = symbols('sigma_theta')



delta_theta  = (a_norm - a_rev)*sin(theta) / (a_norm + a_rev)*cos(theta)

sig_delta_theta = error_probagation(delta_theta, [a_norm, a_rev, theta], [da_norm, da_rev, dtheta])
f_delta_theta = lambdify((a_norm, a_rev, theta), delta_theta)
fsig_delta_theta = lambdify((a_norm, a_rev, theta, da_norm, da_rev, dtheta), sig_delta_theta)

display(sig_delta_theta)

sqrt(sigma_anorm**2*(-(a_norm - a_rev)*sin(theta)*cos(theta)/(a_norm + a_rev)**2 + sin(theta)*cos(theta)/(a_norm + a_rev))**2 + sigma_arev**2*(-(a_norm - a_rev)*sin(theta)*cos(theta)/(a_norm + a_rev)**2 - sin(theta)*cos(theta)/(a_norm + a_rev))**2 + sigma_theta**2*(-(a_norm - a_rev)*sin(theta)**2/(a_norm + a_rev) + (a_norm - a_rev)*cos(theta)**2/(a_norm + a_rev))**2)

In [13]:
# Calculating the gravitational acceleration BALL ON INCLINE

a, theta_ball, del_theta, D_ball, d_rail, = symbols('a, theta_ball, del_theta, D_ball, d_rail')
da, dtheta_ball, d_deltheta, dDball, dd_rail = symbols('sigma_a, sigma_theta_ball, sigma_del_theta, sigma_D_ball, sigma_d_rail')

g_ball = a/(sin(theta_ball + del_theta)) * (1 + 2/5 * (D_ball**2/(D_ball**2 - d_rail**2)))

par_ball = [a, theta_ball, del_theta, D_ball, d_rail]
sig_ball = [da, dtheta_ball, d_deltheta, dDball, dd_rail]

sig_g_ball = error_probagation(g_ball, par_ball, sig_ball)
f_g_ball = lambdify((a, theta_ball, del_theta, D_ball, d_rail), g_ball)
fsig_g_ball = lambdify((a, theta_ball, del_theta, D_ball, d_rail, da, dtheta_ball, d_deltheta, dDball, dd_rail), sig_g_ball)

display(sig_g_ball)

sqrt(0.64*D_ball**4*a**2*d_rail**2*sigma_d_rail**2/((D_ball**2 - d_rail**2)**4*sin(del_theta + theta_ball)**2) + 0.64*a**2*sigma_D_ball**2*(-D_ball**3/(D_ball**2 - d_rail**2)**2 + D_ball/(D_ball**2 - d_rail**2))**2/sin(del_theta + theta_ball)**2 + a**2*sigma_del_theta**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2*cos(del_theta + theta_ball)**2/sin(del_theta + theta_ball)**4 + a**2*sigma_theta_ball**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2*cos(del_theta + theta_ball)**2/sin(del_theta + theta_ball)**4 + sigma_a**2*(0.4*D_ball**2/(D_ball**2 - d_rail**2) + 1)**2/sin(del_theta + theta_ball)**2)

In [14]:
### PERIOD MEASUREMENTS

periods = np.array([2.72, 2.720, 2.722, 2.722])
error_periods = np.array([0.001, 0.001, 0.002, 0.001])

Weighted_mean_periods = np.sum(periods/error_periods**2) / np.sum(1/error_periods**2)
Weighted_error_periods = np.sqrt(1 / np.sum(1/error_periods**2))

chi2_periods = (periods[0] - Weighted_mean_periods)**2/error_periods[0]**2 + (periods[1] - Weighted_mean_periods)**2/error_periods[1]**2 + (periods[2] - Weighted_mean_periods)**2/error_periods[2]**2 + (periods[3] - Weighted_mean_periods)**2/error_periods[3]**2
N_freedom = 2 - 1
chi2_prob_periods = stats.chi2.sf(chi2_periods, N_freedom)
print("weighted mean = ", Weighted_mean_periods, "+-", Weighted_error_periods)
print('Chi2 = ', chi2_periods)
print('P-value = ', chi2_prob_periods)

weighted mean =  2.7207692307692306 +- 0.0005547001962252291
Chi2 =  3.0769230769224
P-value =  0.07941062599897565


In [19]:
# Calculating the gravitational acceleration PENDELUM

# Define symbolic variables
L, T, dL, dT = symbols('L, T, sigma_L, sigma_T')

# Create the expression arcsin(a/b)
g_pendul = L * (2*np.pi/T)**2

par_pendul = [L, T]
sig_pendul = [dL, dT]

sig_g_pendul = error_probagation(g_pendul, par_pendul, sig_pendul)
f_g_pendul = lambdify((L, T), g_pendul)
fsig_g_pendul = lambdify((L, T, dL, dT), sig_g_pendul)

v_L = Weighted_mean
v_dL = Weighted_error
v_T = Weighted_mean_periods
v_dT = Weighted_error_periods

print(f"g_pendulum = {f_g_pendul(v_L, v_T)} +- {fsig_g_pendul(v_L, v_T, v_dL, v_dT)}")

g_pendulum = 9.806555410499127 +- 0.004168237558368737
